# 图像卷积的手写实现

In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import argparse
import cv2

## 卷积运算
+ 图像的某一通道采用"行 x 列"的矩阵存储，对应图像的"高 x 宽"
+ 卷积核kernel为一个方阵，dimension是奇数，这样才会有卷积中心
+ 对原始图像进行padding保证卷积后feature map尺寸不变，注意不同的padding方式，这里使用replicate padding

In [2]:
def convolve(image, kernel):
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]

    pad = (kW - 1) // 2
    print("kW:", kW)
    print("pad:", pad)
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    '''为了方便后面的量化，采用float型，量化完成后再转换成整形'''
    output = np.zeros((iH, iW), dtype="float")
    
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            '''卷积核的卷积ROI区域'''
            roi = image[(y - pad):(y + pad +1), (x - pad):(x + pad + 1)]
            k = (roi * kernel).sum()
            output[(y - pad), (x - pad)] = k

    '''量化
    pixel的intensity是[0, 255]，经过卷积运算后可能超出这一范围，
    我们需要卷积后的结果约束在这一范围内，对超出范围的数据进行截断，譬如[18, 208, 298]-->[18, 208, 255]
    这里"out_range"是image的dtype，也即"float"类型，范围为[0, 1.0]
    然后将截断后的output数据按照[0, 255]-->[0, 1.0]的映射关系进行映射量化
    再量化回[0, 255]范围（[0, 1.0]-->[0, 255]），且转换为整形
    '''
    output = rescale_intensity(output, in_range=(0, 255))
    outpt = (output * 255).astype("uint8")
    
    return output

In [3]:
'''
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
               help="path to the input image")
args = vars(ap.parse_args())
'''

'\nap = argparse.ArgumentParser()\nap.add_argument("-i", "--image", required=True,\n               help="path to the input image")\nargs = vars(ap.parse_args())\n'

## 各种样式的卷积核

In [4]:
smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))
largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))

sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]), dtype="int")

laplacian = np.array((
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]), dtype="int")

sobelX = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype="int")
sobelY = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype="int")

emboss = np.array((
    [-2, -1, 0],
    [-1, 1, 1],
    [0, 1, 2]), dtype="int")

kernelBank = (
    ("small_blur", smallBlur),
    ("large_blur", largeBlur),
    ("sharpen", sharpen),
    ("laplacian", laplacian),
    ("sobel_x", sobelX),
    ("sobel_y", sobelY),
    ("emboss", emboss))

### 注意：
OpenCV中默认的彩图通道排列是BGR，历史原因

In [5]:
image = cv2.imread("./beauty.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

for (kernelName, kernel) in kernelBank:
    print("[INFO] applying {} kernel...".format(kernelName))
    convolveOutput = convolve(gray, kernel)
    opencvOutpt = cv2.filter2D(gray, -1 , kernel)
    
    cv2.imshow("Original", gray)
    cv2.imshow("{} - convolve".format(kernelName), convolveOutput)
    cv2.imshow("{} - opencv".format(kernelName), opencvOutpt)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[INFO] applying small_blur kernel...
kW: 7
pad: 3
[INFO] applying large_blur kernel...
kW: 21
pad: 10
[INFO] applying sharpen kernel...
kW: 3
pad: 1
[INFO] applying laplacian kernel...
kW: 3
pad: 1
[INFO] applying sobel_x kernel...
kW: 3
pad: 1
[INFO] applying sobel_y kernel...
kW: 3
pad: 1
[INFO] applying emboss kernel...
kW: 3
pad: 1
